In [10]:
import pandas as pd
import sqlite3 as sq
from datetime import datetime

In [11]:
rs_stocks = pd.read_csv('../src/rs_stocks.csv')
rs_stocks

,Rank,Ticker,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,1,FNGR,Communication Services,Telecom Services,NASDAQ,586.60,99,99,95,33
1,2,CABA,Healthcare,Biotechnology,NASDAQ,483.88,99,99,99,99
2,3,CVNA,Consumer Cyclical,Internet Retail,NYSE,422.53,99,99,99,27
3,4,MLTX,Healthcare,Biotechnology,NASDAQ,385.99,99,99,99,99
4,5,AAOI,Technology,Semiconductors,NASDAQ,374.71,99,99,98,58
...,...,...,...,...,...,...,...,...,...,...
6613,6614,AULT,Industrials,Aerospace & Defense,NYSE MKT,3.63,0,0,0,5
6614,6615,GMBL,Consumer Cyclical,Gambling,NASDAQ,3.54,0,0,3,0
6615,6616,SMX,Industrials,Specialty Business Services,NASDAQ,3.52,0,0,0,0
6616,6617,AVTX,Healthcare,Biotechnology,NASDAQ,3.05,0,0,74,3


In [12]:
mkt_cap_data = pd.read_csv('../src/TV.csv')
mkt_cap_data

,Ticker,Description,Market Capitalization,Upcoming Earnings Date
0,AAPL,Apple Inc.,2.733333e+12,2023-10-26
1,MSFT,Microsoft Corporation,2.490234e+12,2023-10-25
2,GOOG,Alphabet Inc.,1.722943e+12,2023-10-24
3,GOOGL,Alphabet Inc.,1.722692e+12,2023-10-24
4,AMZN,"Amazon.com, Inc.",1.478825e+12,2023-10-26
...,...,...,...,...
17819,TMDV,ProShares Russell US Dividend Growers ETF,NaN,NaN
17820,USO,United States Oil Fund,NaN,NaN
17821,LTPZ,Pimco 15 Year U.S. TIPS Index Exchange-Traded ...,NaN,NaN
17822,BALT,Innovator Defined Wealth Shield ETF,NaN,NaN


In [13]:
# Load the 'price_action' table from the 'market_data.db' database into a DataFrame named 'price_action'.
conn = sq.connect('../market_data.db')
price_action = pd.read_sql_query("SELECT * FROM price_action", conn)
conn.close()
price_action

,date,ticker,open,high,low,close,adjclose,volume
0,2020-01-02,AAOI,12.1300,12.5300,11.8000,12.5000,12.5000,885000.0
1,2020-01-03,AAOI,12.2500,12.3250,12.0100,12.1200,12.1200,498400.0
2,2020-01-06,AAOI,12.0000,12.2000,11.8600,12.1500,12.1500,361600.0
3,2020-01-07,AAOI,12.2000,12.4650,12.0550,12.4300,12.4300,330500.0
4,2020-01-08,AAOI,12.4100,12.8200,12.3700,12.7000,12.7000,520600.0
...,...,...,...,...,...,...,...,...
5437795,2023-09-12,ALLR,1.0300,1.1000,1.0300,1.0800,1.0800,112046.0
5437796,2023-09-12,AULT,0.7757,0.7757,0.6950,0.7130,0.7130,781717.0
5437797,2023-09-12,GMBL,0.1115,0.1233,0.1115,0.1150,0.1150,2555979.0
5437798,2023-09-12,SMX,1.4200,1.4400,1.3700,1.3700,1.3700,41956.0


In [14]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_full(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Price'] >= row['52Week-Low'] * 1.25,
        row['Price'] >= row['52Week-High'] * 0.85,
        row['Percentile'] >= 70,
        row['SMA200_1M_Trend'] == 1,
        row['Avg_Vol_90d'] > 100000,
        row['Avg_Vol_10d'] <= row['Avg_Vol_30d']

    ]
    return 'yes' if all(criteria) else 'no'

In [15]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_basic(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Percentile'] >= 70,
        row['Avg_Vol_90d'] > 100000,

    ]
    return 'yes' if all(criteria) else 'no'

In [16]:
# Define the PowerTrend criteria
def power_trend(row, prev_power_trend):
    # Entry criteria
    if (
        row['lows_above_EMA21'] >= 10 and
        row['EMA21_above_SMA50'] >= 5 and
        row['SMA50_uptrend'] == 1 and
        row['close_above_open'] == 1
    ):
        return 'in'
    
    # Exit criteria
    if prev_power_trend == 'in' and (
        row['EMA21'] < row['SMA50'] or
        (row['close'] < row['high_last_week'] * 0.9 and row['close'] < row['SMA50'])
    ):
        return 'out'
    
    return prev_power_trend

In [17]:
screen_df =  pd.merge(price_action, mkt_cap_data, left_on='ticker', right_on='Ticker', how='left')
screen_df = pd.merge(screen_df, rs_stocks, left_on='Ticker', right_on='Ticker', how='left')
screen_df

,date,ticker,open,high,low,close,adjclose,volume,Ticker,Description,...,Upcoming Earnings Date,Rank,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,2020-01-02,AAOI,12.1300,12.5300,11.8000,12.5000,12.5000,885000.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,5.0,Technology,Semiconductors,NASDAQ,374.71,99.0,99.0,98.0,58.0
1,2020-01-03,AAOI,12.2500,12.3250,12.0100,12.1200,12.1200,498400.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,5.0,Technology,Semiconductors,NASDAQ,374.71,99.0,99.0,98.0,58.0
2,2020-01-06,AAOI,12.0000,12.2000,11.8600,12.1500,12.1500,361600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,5.0,Technology,Semiconductors,NASDAQ,374.71,99.0,99.0,98.0,58.0
3,2020-01-07,AAOI,12.2000,12.4650,12.0550,12.4300,12.4300,330500.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,5.0,Technology,Semiconductors,NASDAQ,374.71,99.0,99.0,98.0,58.0
4,2020-01-08,AAOI,12.4100,12.8200,12.3700,12.7000,12.7000,520600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,5.0,Technology,Semiconductors,NASDAQ,374.71,99.0,99.0,98.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5437795,2023-09-12,ALLR,1.0300,1.1000,1.0300,1.0800,1.0800,112046.0,ALLR,"Allarity Therapeutics, Inc.",...,2023-11-14,6613.0,Healthcare,Biotechnology,NASDAQ,5.13,0.0,0.0,0.0,0.0
5437796,2023-09-12,AULT,0.7757,0.7757,0.6950,0.7130,0.7130,781717.0,AULT,"Ault Alliance, Inc.",...,2023-11-22,6614.0,Industrials,Aerospace & Defense,NYSE MKT,3.63,0.0,0.0,0.0,5.0
5437797,2023-09-12,GMBL,0.1115,0.1233,0.1115,0.1150,0.1150,2555979.0,GMBL,Esports Entertainment Group Inc.,...,2023-10-12,6615.0,Consumer Cyclical,Gambling,NASDAQ,3.54,0.0,0.0,3.0,0.0
5437798,2023-09-12,SMX,1.4200,1.4400,1.3700,1.3700,1.3700,41956.0,SMX,SMX (Security Matters) Public Limited Company,...,NaN,6616.0,Industrials,Specialty Business Services,NASDAQ,3.52,0.0,0.0,0.0,0.0


In [18]:
# Add the 'Price' column representing the latest close for each stock
screen_df['Price'] = screen_df.groupby('Ticker')['close'].transform('last')

# Calculate the 50-day Simple Moving Average (SMA50) for each stock.
screen_df['SMA50'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=50).mean())

# Calculate the 21-day Exponential Moving Average (EMA21) for each stock.
screen_df['EMA21'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.ewm(span=21, adjust=False).mean())

# Calculate the 150-day Simple Moving Average (SMA150) for each stock.
screen_df['SMA150'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=150).mean())

# Calculate the 200-day Simple Moving Average (SMA200) for each stock.
screen_df['SMA200'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=200).mean())

# Calculate the 90-day Average Volume for each stock.
screen_df['Avg_Vol_90d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=90).mean())

# Calculate the 30-day Average Volume for each stock.
screen_df['Avg_Vol_30d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=30).mean())

# Calculate the 10-day Average Volume for each stock.
screen_df['Avg_Vol_10d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=10).mean())

# Calculate the 52-week high (highest closing price in the last 252 trading days) for each stock.
screen_df['52Week-High'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).max())

# Calculate the 52-week low (lowest closing price in the last 252 trading days) for each stock.
screen_df['52Week-Low'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).min())

# Calculate the price performance over the last 252 trading days
screen_df['12M Return'] = screen_df.groupby('Ticker')['close'].pct_change(periods=252)

# Calculate the highest high and lowest low over the last 5 daysb g
screen_df['10D_high'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=10).max())
screen_df['10D_low'] = screen_df.groupby('Ticker')['low'].transform(lambda x: x.rolling(window=10).min())
# Calculate the range as a percentage of the highest highb g
screen_df['10D_range_pct'] = (screen_df['10D_high'] - screen_df['10D_low']) / screen_df['10D_high'] * 100

# Identify low-risk entries
screen_df['Low-Risk_Entry'] = (screen_df['10D_range_pct'] < 8).astype(int)

# Calculate the SMA200 trend over the last month
screen_df['SMA200_1M_Trend'] = screen_df.groupby('Ticker')['SMA200'].transform(lambda x: x.diff(periods=21)).gt(0).astype(int)

# Calculate criteria for PowerTrend
screen_df['lows_above_EMA21'] = screen_df.groupby('Ticker').apply(lambda x: (x['low'] > x['EMA21']).rolling(window=10).sum()).reset_index(level=0, drop=True)
screen_df['EMA21_above_SMA50'] = screen_df.groupby('Ticker').apply(lambda x: (x['EMA21'] > x['SMA50']).rolling(window=5).sum()).reset_index(level=0, drop=True)
screen_df['SMA50_uptrend'] = screen_df.groupby('Ticker')['SMA50'].transform(lambda x: x.diff().rolling(window=21).sum()).gt(0).astype(int)
screen_df['close_above_open'] = (screen_df['close'] > screen_df['open']).astype(int)
screen_df['high_last_week'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=5).max())

# Apply the Minervini criteria to each row
screen_df['Minervini_basic'] = screen_df.apply(minervini_criteria_basic, axis=1)
screen_df['Minervini_full'] = screen_df.apply(minervini_criteria_full, axis=1)

# Apply the PowerTrend criteria to each row
prev_power_trend = 'out'
power_trends = []

for _, row in screen_df.iterrows():
    current_power_trend = power_trend(row, prev_power_trend)
    power_trends.append(current_power_trend)
    prev_power_trend = current_power_trend
    
# Apply the Minervini criteria to each row
screen_df['PowerTrend'] = power_trends

# Create the 'screen_df' DataFrame with the calculated metrics.
metrics = ['Ticker', 'EMA21', 'SMA50', 'SMA150', 'SMA200', 'Avg_Vol_10d', 'Avg_Vol_30d', 'Avg_Vol_90d', '52Week-High', 
           '52Week-Low', 'Minervini_basic', 'Minervini_full', 'PowerTrend', 'Low-Risk_Entry', 'Market Capitalization',
           'Upcoming Earnings Date', 'Industry', 'Percentile', '1 Month Ago', '3 Months Ago', '6 Months Ago']
screen_df = screen_df.drop_duplicates(subset='Ticker', keep='last')[metrics]
screen_df

,Ticker,EMA21,SMA50,SMA150,SMA200,Avg_Vol_10d,Avg_Vol_30d,Avg_Vol_90d,52Week-High,52Week-Low,...,Minervini_full,PowerTrend,Low-Risk_Entry,Market Capitalization,Upcoming Earnings Date,Industry,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
55375,NSTC,10.368905,10.363360,10.226553,10.175265,25030.0,8.436667e+03,6.910000e+03,11.550000,9.8100,...,no,out,1,1.461616e+08,NaN,NaN,NaN,NaN,NaN,NaN
90093,ELC,0.425458,NaN,NaN,NaN,4340.0,NaN,NaN,NaN,NaN,...,no,out,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104997,SBIG,0.249772,0.335100,0.549633,0.593415,149720.0,1.681933e+05,2.090022e+05,1.740000,0.2000,...,no,out,0,6.653174e+06,2023-11-13,NaN,NaN,NaN,NaN,NaN
106496,HLGN,0.222220,0.244512,0.287427,0.407976,7598999.5,2.937404e+06,2.074079e+06,2.450000,0.1789,...,no,out,0,2.724860e+07,2023-11-06,NaN,NaN,NaN,NaN,NaN
610264,FBGX,601.693715,602.626383,526.063824,496.945348,240.0,1.733333e+02,1.900000e+02,639.307983,356.0000,...,no,out,1,NaN,NaN,NaN,91.0,90.0,90.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5437795,ALLR,1.695330,2.475800,62.905267,153.165950,1018574.6,4.626482e+05,5.690082e+05,1764.000000,1.0800,...,no,out,0,2.810688e+06,2023-11-14,Biotechnology,0.0,0.0,0.0,0.0
5437796,AULT,1.400660,2.602060,17.258953,22.499215,2795011.7,1.190107e+06,8.712929e+05,84.000000,0.7130,...,no,out,0,3.794383e+06,2023-11-22,Aerospace & Defense,0.0,0.0,0.0,5.0
5437797,GMBL,0.181902,0.430620,1.924073,3.936055,7106957.9,1.273681e+07,4.388730e+06,47.500000,0.1140,...,no,out,0,5.442339e+06,2023-10-12,Gambling,0.0,0.0,3.0,0.0
5437798,SMX,1.891873,2.529900,38.829673,84.727805,411675.6,2.201518e+05,2.743674e+05,321.200012,1.3700,...,no,out,0,3.109793e+06,NaN,Specialty Business Services,0.0,0.0,0.0,0.0


In [19]:
today = datetime.now().strftime('%Y-%m-%d')
nome_arquivo = f'../output/screen_{today}.xlsx'
screen_df.to_excel(nome_arquivo, engine='xlsxwriter')

In [20]:
# Get the current date
current_date = datetime.now().strftime('%d-%m-%Y')
table_name = f"screen-{current_date}"

# Connect to the SQLite database
conn = sq.connect('../market_data.db')

# Save the screen_df to the database with the table name based on the current date
screen_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()